# Run parcel tracking with tracer

In [1]:
import numpy as np
from datetime import timedelta as delta
from datetime import datetime
import xarray as xr
from parcels import FieldSet, Variable, ParticleSet, JITParticle, AdvectionRK4, AdvectionRK4_3D, ErrorCode, Field
import glob

%matplotlib inline

#### Parameters

In [2]:
year = 2002
land_color = '#a9a7a2'
jmin, jmax = 159, 799
imin, imax = 1139, 2179

#### Load files

In [3]:
# Open particle initialization location file:
ini_loc = xr.open_dataset('/ocean/brogalla/GEOTRACES/parcels/Pb-chapter/western-PC-transect-parcels.nc')
parcel_lons = ini_loc['longitude'].values
parcel_lats = ini_loc['latitude'].values
parcel_deps = ini_loc['depth'].values

#### General setup

In [16]:
# Mesh mask
mesh_mask = '/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc'

# ANHA12 NEMO grid U,V,W files
folder_ANHA12      = '/data/brogalla/ANHA12/'
ANHA12_gridU_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_y{year}m??d??_gridU.nc'))[0:20]
ANHA12_gridV_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_y{year}m??d??_gridV.nc'))[0:20]
ANHA12_gridW_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_5d_gridW_y{year}m??d??.nc'))[0:20]

# ANHA12 NEMO Pb model files
folder_Pb = f'/data/brogalla/run_storage/Pb-new-ini2-20220922/'
files_Pb  = sorted(glob.glob(f'{folder_Pb}ANHA12_EXH006_5d_{year}0101_{year}1231_ptrc_T_{year}*'))[0:20]

# Create timestamps from file dates:
time_stamps = []
for n, file in enumerate(ANHA12_gridU_files):
    dateU  = datetime.strptime(file[36:47],'y%Ym%md%d')
    dateV  = datetime.strptime(ANHA12_gridV_files[n][36:47],'y%Ym%md%d')
    dateW  = datetime.strptime(ANHA12_gridW_files[n][45:56],'y%Ym%md%d')
    datePb = datetime.strptime(files_Pb[n][99:107],'%Y%m%d')
    
    if (dateU != dateV) | (dateU != dateW) | (dateU != datePb):
        print('Dates not equal!')
    
    # double-check that the date is the same for U, V, and W files
    time_stamps.append(np.datetime64(f'{dateU.year:02}-{dateU.month:02}-{dateU.day:02}'))

timestamps = np.expand_dims(np.array(time_stamps), axis=1)

In [49]:
f1 = xr.open_dataset(ANHA12_gridU_files[0])
f2 = xr.open_dataset(files_Pb[0])

In [53]:
f2

<xarray.Dataset>
Dimensions:               (y: 1040, x: 640, deptht: 50, axis_nbounds: 2,
                           time_counter: 1)
Coordinates:
    nav_lat               (y, x) float32 ...
    nav_lon               (y, x) float32 ...
  * deptht                (deptht) float32 0.494 1.541 ... 5.275e+03 5.728e+03
    time_centered         (time_counter) object 2002-01-03 12:00:00
  * time_counter          (time_counter) object 2002-01-03 12:00:00
Dimensions without coordinates: y, x, axis_nbounds
Data variables:
    deptht_bounds         (deptht, axis_nbounds) float32 0.0 1.011 ... 5.958e+03
    dissolpb              (time_counter, deptht, y, x) float32 ...
    time_centered_bounds  (time_counter, axis_nbounds) object 2002-01-01 00:0...
    time_counter_bounds   (time_counter, axis_nbounds) object 2002-01-01 00:0...
Attributes:
    name:         ANHA12_EXH006_5d_20020101_20021231
    description:  ocean T grid passive tracers
    title:        ocean T grid passive tracers
    Conventions:  CF-1.6
    timeStamp:    2022-Sep-23 01:19:40 GMT
    uuid:         c661def2-7d02-489f-80e4-a1523b2e3998

In [83]:
filenames = {'U': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridU_files},
             'V': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridV_files},
             'W': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridW_files},
             'Pb': {'lon': files_Pb[0], 'lat': files_Pb[0], 'depth': files_Pb[0], 'data': files_Pb}}

dimensions = {'U': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'V': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'W': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'Pb': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'deptht'}}

variables = {'U': 'vozocrtx',
             'V': 'vomecrty',
             'W': 'vovecrtz',
             'Pb': 'dissolpb'}

indices_subset = {'U':  {'lon': range(jmin, jmax), 'lat': range(imin, imax), 'depth': range(0,50)},
                  'V':  {'lon': range(jmin, jmax), 'lat': range(imin, imax), 'depth': range(0,50)},
                  'W':  {'lon': range(jmin, jmax), 'lat': range(imin, imax), 'depth': range(0,50)},
                  'Pb': {'lon': range(0,640)     , 'lat': range(0,1040)    , 'depth': range(0,50)}}

# needed to provide time as timestamps in order to avoid time origin type issue
fieldset  = FieldSet.from_nemo(filenames, variables, dimensions, \
                               timestamps=timestamps, \
                               allow_time_extrapolation=False, \
                               indices=indices_subset)

In [84]:
# print(fieldset.V.indices)
print(fieldset.W.depth.max())
print(fieldset.Pb.depth.max())

5500.0015
5727.9165


#### Classes and functions

In [85]:
# define new particle class for Pb
class PbParticle(JITParticle): 
    dissolvedPb = Variable('dissolpb', initial=fieldset.Pb)  # Variable 'dissolpb' initialised by sampling the dissolved Pb field

In [86]:
def DeleteParticle(particle, fieldset, time):
    print("Particle [%d] lost !! (%g %g %g %g)" % (particle.id, particle.lon, particle.lat, particle.depth, particle.time))
    particle.delete()

In [87]:
def SamplePb(particle, fieldset, time):
         particle.dissolpb = fieldset.Pb[particle.time, particle.depth, particle.lat, particle.lon]

#### Run

With Pb:

In [88]:
# When do I want to start tracking the parcels?

release_time  = 0

In [89]:
print(np.amin(fieldset.U.lon), np.amax(fieldset.U.lon))
print(np.amin(fieldset.V.lon), np.amax(fieldset.V.lon))
print(np.amin(fieldset.W.lon), np.amax(fieldset.W.lon))
print(np.amin(fieldset.Pb.lon), np.amax(fieldset.Pb.lon))

print(np.amin(fieldset.U.lat), np.amax(fieldset.U.lat))
print(np.amin(fieldset.V.lat), np.amax(fieldset.V.lat))
print(np.amin(fieldset.W.lat), np.amax(fieldset.W.lat))
print(np.amin(fieldset.Pb.lat), np.amax(fieldset.Pb.lat))

print(np.amin(fieldset.U.depth), np.amax(fieldset.U.depth))
print(np.amin(fieldset.V.depth), np.amax(fieldset.V.depth))
print(np.amin(fieldset.W.depth), np.amax(fieldset.W.depth))
print(np.amin(fieldset.Pb.depth), np.amax(fieldset.Pb.depth))

-176.72366 -32.57282
-176.72366 -32.57282
-176.72366 -32.57282
-176.72366 -32.57282
55.4229 85.85598
55.4229 85.85598
55.4229 85.85598
55.4229 85.85598
0.0 5500.0015
0.0 5500.0015
0.0 5500.0015
0.49402538 5727.9165


double-checked that the time is being read in correctly. Next, check that the Pb concentrations are more or less correct for the location sampled. Subset indexing seems to be correct when I look at the min and max of lon and lat

In [90]:
release_lon   = [-130, -145, -70, -55] # east
release_lat   = [75, 74, 75, 65]  # north
release_depth = [1, 1, 1, 1] 
# release_time  = fieldset.U.grid.time[0]
pset = ParticleSet(fieldset=fieldset, pclass=PbParticle, lon=release_lon, lat=release_lat, time=0, depth=release_depth)

In [91]:
# pset = ParticleSet(fieldset = fieldset, 
#                    pclass   = PbParticle, 
#                    lon      = parcel_lons, 
#                    lat      = parcel_lats, 
#                    depth    = parcel_deps,
#                    time     = release_time)

# release_lon   = [-130, -145, -70, -55] # east
# release_lat   = [75, 74, 75, 65]  # north
# release_depth = [1, 1, 1, 1] 
# release_time  = 0
# pset = ParticleSet(fieldset=fieldset, pclass=PbParticle, lon=release_lon, lat=release_lat, time=release_time, depth=release_depth)

output_file = pset.ParticleFile(name="/ocean/brogalla/GEOTRACES/parcels/Pb-chapter/Pb-test2.zarr", outputdt=delta(hours=12))

# particles are bumping against land and being deleted, if I instead just want them to keep moving:
pset.execute(AdvectionRK4_3D + pset.Kernel(SamplePb), 
             runtime     = delta(hours=24*10), 
             dt          = delta(hours=2),
             output_file = output_file,
             recovery    = {ErrorCode.ErrorOutOfBounds: DeleteParticle})

# dt      --- the timestep of the kernel
# runtime --- the total length of the run

INFO: Compiled ArrayPbParticleAdvectionRK4_3DSamplePb ==> /tmp/parcels-2672/libb6ac7b3ba43e020eb21721d7d8090f3d_0.so


Particle [36] lost !! (-130 75 1 0)
Particle [37] lost !! (-145 74 1 0)


INFO: Output files are stored in /ocean/brogalla/GEOTRACES/parcels/Pb-chapter/Pb-test2.zarr.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 864000.0/864000.0 [00:05<00:00, 151050.69it/s]


Without Pb: (to check release locations)

In [7]:
release_time  = 0

In [9]:
pset = ParticleSet(fieldset = fieldset_vel, 
                   lon      = parcel_lons, 
                   lat      = parcel_lats, 
                   depth    = parcel_deps,
                   time     = release_time)

output_file = pset.ParticleFile(name="/ocean/brogalla/GEOTRACES/parcels/Pb-chapter/2002-parcel-tracks.zarr", outputdt=delta(hours=12))

pset.execute(AdvectionRK4_3D, 
             runtime     = delta(hours=24*90), 
             dt          = delta(hours=2),
             recovery    = {ErrorCode.ErrorOutOfBounds: DeleteParticle},
             output_file = output_file)

# dt      --- the timestep of the kernel
# runtime --- the total length of the run

INFO: Compiled ArrayJITParticleAdvectionRK4_3D ==> /tmp/parcels-2672/lib21460d5b6a659cb3c66e578d3c508f68_0.so
INFO: Output files are stored in /ocean/brogalla/GEOTRACES/parcels/Pb-chapter/2002-parcel-tracks.zarr.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7776000.0/7776000.0 [14:18<00:00, 9053.48it/s]


AttributeError: 'ParticleFileSOA' object has no attribute 'export'